# Sistema RAG para Documento Bre-B


## Configuración

Este notebook implementa un sistema básico de Recuperación Aumentada por Generación (RAG) utilizando componentes de LangChain y modelos de Hugging Face. Es un punto de partida para entender el flujo RAG.



### Pasos Previos y Configuración del Entorno

Para ejecutar este cuaderno, necesitas tener **Python** (se recomienda Python 3.9 o superior) y, preferiblemente, **Conda** para una gestión de entorno aislada y eficiente.


#### Crear y Activar un Entorno Conda (Recomendado)

Si tienes Conda, es la mejor manera de asegurar que todas las dependencias se instalen correctamente sin conflictos con otros proyectos.

```bash
# Abre tu terminal o Anaconda Prompt y ejecuta:
conda create -n rag-current-env python=3.9 -y
conda activate rag-current-env

### Instalación de Librerías Necesarias

Asegúrate de tener estas librerías instaladas en tu entorno.

In [29]:
!pip install langchain
!pip install transformers
!pip install torch 
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -U langchain-community

### Importar librerías

In [20]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS # O Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


# 1. Obtener ruta del documento a cargar

In [30]:
import os
print(os.getcwd())

/Users/dannybarrientos/Workspace/ia-wompi/rag-iaw/notebooks


# 2. Carga del Documento

Cargamos el contenido del archivo "bre_b.txt" desde la ruta especificada en la estructura del proyecto.
La ruta del archivo se ajusta a la estructura Cookiecutter: data/raw/bre_b.txt

In [23]:
# La ruta del archivo se ajusta a la estructura del proyecto.
# Asegúrate de que el archivo 'bre-b.txt' esté en la ubicación correcta.
file_path = "../data/raw/bre-b.txt"

try:
    # `TextLoader` es la clase de LangChain para leer archivos de texto.
    # Usamos `encoding='latin-1'` para manejar caracteres especiales.
    loader = TextLoader(file_path, encoding='latin-1')
    documents = loader.load() # Carga el contenido en la variable `documents`.
    print(f"Documento '{file_path}' cargado exitosamente. Número de páginas/documentos: {len(documents)}")
except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no se encontró. Asegúrate de que esté en la ubicación correcta.")
except UnicodeDecodeError as e:
    print(f"Error de codificación al leer el archivo: {e}")
    print("Intenta cambiar la codificación (por ejemplo, a 'latin-1' o 'cp1252').")
except Exception as e:
    print(f"Ocurrió un error inesperado durante la carga del documento: {e}")

Documento '../data/raw/bre-b.txt' cargado exitosamente. Número de páginas/documentos: 1


# 3. Segmentación del Texto (Chunking)

Dividimos el documento en "chunks" más pequeños para un mejor procesamiento.
Esto ayuda a que el modelo de embedding sea más preciso y a que el LLM maneje el contexto.

In [24]:
# `RecursiveCharacterTextSplitter` divide el texto de forma inteligente.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Cada fragmento tendrá un máximo de 1000 caracteres.
    chunk_overlap=100,    # Los fragmentos se superpondrán en 100 caracteres para mantener el contexto.
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(documents) # Realiza la segmentación.
print(f"\nDocumento segmentado en {len(chunks)} chunks.")


Documento segmentado en 34 chunks.


# 4. Generación de Embeddings y Creación del Vector Store
---------------------------------------------------
Convertimos los chunks de texto en embeddings (vectores numéricos) y los almacenamos en un vector store (FAISS) para búsqueda de similitud.
Se utiliza un modelo de embeddings pre-entrenado de Hugging Face.
Este modelo se descargará la primera vez que se ejecute.

In [25]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Creamos el vector store FAISS a partir de los chunks y los embeddings.
# FAISS es una librería para búsqueda eficiente de similitud en grandes bases de datos de vectores.
db = FAISS.from_documents(chunks, embeddings)

print("\nEmbeddings generados y FAISS vector store creado exitosamente.")

/opt/homebrew/anaconda3/envs/rag-env/lib/python3.9/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Embeddings generados y FAISS vector store creado exitosamente.


# 5. Uso de un LLM (Hugging Face Pipeline)
---------------------------------------
Configuramos el Large Language Model (LLM) que usaremos para generar las respuestas

In [26]:
print("\nConfigurando el Large Language Model (LLM)...")

model_id = "google/flan-t5-small"

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline 

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,
                                             torch_dtype=torch.float32)

device = 0 if torch.cuda.is_available() else -1
print(f"Usando dispositivo: {'GPU' if device == 0 else 'CPU'}")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    device=device
)

llm = HuggingFacePipeline(pipeline=pipe)

print("LLM configurado exitosamente.")


Configurando el Large Language Model (LLM)...


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Usando dispositivo: CPU
LLM configurado exitosamente.


# Sección para Realizar Preguntas al Documento
   ---------------------------------------------
Esta sección permitirá interactuar con el sistema, hacer preguntas y obtener respuestas
con referencias a los chunks originales.


In [ ]:
print("\n--- ¡Sistema RAG listo para recibir preguntas! ---")
print("Escribe 'salir' en cualquier momento para terminar.\n")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(), # Permite a LangChain buscar en el vector store
    return_source_documents=True # Importante para obtener los chunks que el LLM usó
)

while True:
    query = input("Tu pregunta: ")
    if query.lower() == 'salir':
        print("Saliendo del sistema RAG. ¡Adiós!")
        break

    print("Buscando y generando respuesta...")
    try:
        result = qa_chain({"query": query})

        print("\n**Respuesta:**")
        print(result["result"])

        print("\n**--- Fuentes Utilizadas del Documento ---**")
        if result["source_documents"]:
            for i, doc in enumerate(result["source_documents"]):
                # Imprimimos los primeros 200 caracteres del chunk y su metadata
                print(f"Chunk {i+1}:")
                print(f"  Contenido inicial: \"{doc.page_content[:200]}...\"")
                # Si los documentos tuvieran metadata útil (como número de página, título, etc.),
                # podrías imprimirla aquí:
                # print(f"  Metadata: {doc.metadata}")
                print("-" * 30)
        else:
            print("No se encontraron fuentes específicas para esta respuesta.")
        print("\n" + "=" * 50 + "\n")

    except Exception as e:
        print(f"Ocurrió un error al procesar la pregunta: {e}")
        print("Por favor, intenta de nuevo.")


--- ¡Sistema RAG listo para recibir preguntas! ---
Escribe 'salir' en cualquier momento para terminar.



Tu pregunta:  Que es bre-b?


/var/folders/2c/h9s5jckx3491cqs2ch9zgxv00000gn/T/ipykernel_82338/3438884223.py:27: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})
/opt/homebrew/anaconda3/envs/rag-env/lib/python3.9/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Token indices sequence length is longer than the specified maximum sequence length for this model (1272 > 512). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Buscando y generando respuesta...

**Respuesta:**
Qué valor máximo por transacción en Bre-B es de 1.000 unidades de valor básico (UVB) del ao en curso.

**--- Fuentes Utilizadas del Documento ---**
Chunk 1:
  Contenido inicial: "¿Bre-B será una nueva aplicación para pasar plata?
No, para usar Bre-B no tendrás que descargar ninguna aplicación. En el segundo semestre de 2025 vas a ver una opción dentro de las aplicaciones de la..."
------------------------------
Chunk 2:
  Contenido inicial: "*El valor máximo por transacción en Bre-B es de 1.000 unidades de valor básico (UVB) del año en curso. Para el 2025 es de COP 11.552.

Código QR
Bre-B también tiene códigos QR.
Solo escanee el código ..."
------------------------------
Chunk 3:
  Contenido inicial: "Con Bre-B, sin importar tu entidad financiera, podrás pasar y recibir plata en segundos. Lo único que tendrás que hacer es inscribir tus llaves y elegir a qué cuenta de ahorro, corriente o depósito de..."
------------------------------
C

Tu pregunta:  ¿Bre-B tendrá códigos QR?


...

/opt/homebrew/anaconda3/envs/rag-env/lib/python3.9/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Buscando y generando respuesta...

**Respuesta:**
1.000 unidades de valor básico (UVB) del ao

**--- Fuentes Utilizadas del Documento ---**
Chunk 1:
  Contenido inicial: "*El valor máximo por transacción en Bre-B es de 1.000 unidades de valor básico (UVB) del año en curso. Para el 2025 es de COP 11.552.

Código QR
Bre-B también tiene códigos QR.
Solo escanee el código ..."
------------------------------
Chunk 2:
  Contenido inicial: "¿Bre-B será una nueva aplicación para pasar plata?
No, para usar Bre-B no tendrás que descargar ninguna aplicación. En el segundo semestre de 2025 vas a ver una opción dentro de las aplicaciones de la..."
------------------------------
Chunk 3:
  Contenido inicial: "Con Bre-B, sin importar tu entidad financiera, podrás pasar y recibir plata en segundos. Lo único que tendrás que hacer es inscribir tus llaves y elegir a qué cuenta de ahorro, corriente o depósito de..."
------------------------------
Chunk 4:
  Contenido inicial: "¿Cómo funcionará Bre-B?
Los 

Tu pregunta:  ¿Cuál es la función principal de Bre-B?


/opt/homebrew/anaconda3/envs/rag-env/lib/python3.9/site-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Buscando y generando respuesta...

**Respuesta:**
1.000 unidades de valor básico (UVB) del ao

**--- Fuentes Utilizadas del Documento ---**
Chunk 1:
  Contenido inicial: "*El valor máximo por transacción en Bre-B es de 1.000 unidades de valor básico (UVB) del año en curso. Para el 2025 es de COP 11.552.

Código QR
Bre-B también tiene códigos QR.
Solo escanee el código ..."
------------------------------
Chunk 2:
  Contenido inicial: "¿Bre-B será una nueva aplicación para pasar plata?
No, para usar Bre-B no tendrás que descargar ninguna aplicación. En el segundo semestre de 2025 vas a ver una opción dentro de las aplicaciones de la..."
------------------------------
Chunk 3:
  Contenido inicial: "Con Bre-B, sin importar tu entidad financiera, podrás pasar y recibir plata en segundos. Lo único que tendrás que hacer es inscribir tus llaves y elegir a qué cuenta de ahorro, corriente o depósito de..."
------------------------------
Chunk 4:
  Contenido inicial: "¿Cómo funcionará Bre-B?
Los 